## Workspace Management demo

This notebook presents examples of how to manage and use team workspaces!

In [1]:
import jwt
import requests
import datetime
import boto3
import urllib3
import time
urllib3.disable_warnings()

In [2]:
realm = "eoepca"
base_domain = "apx.develop.eoepca.org"
keycloak_endpoint = f"https://iam-auth.{base_domain}"
workspaces_endpoint = f'https://workspace-api.{base_domain}/workspaces'
token_endpoint = f"{keycloak_endpoint}/realms/{realm}/protocol/openid-connect/token"
minio_endpoint = "https://minio.develop.eoepca.org"

### token handling

In [3]:
def get_user_token(username, password):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "scope": "roles",
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "demo",
        "client_secret": "demo"
    }    
    response = requests.post(token_endpoint, headers=headers, data=data)
    if response.ok:
        return response.json()["access_token"]
    else:
        print(response)
        return None

In [4]:
#the following example users are expected to exist
token_user1 = get_user_token("example-user-1", "changeme")
decoded_token_user1 = jwt.decode(token_user1, options={"verify_signature": False})
print(f"\n{decoded_token_user1}")
token_user2 = get_user_token("example-user-2", "changeme")
decoded_token_user2 = jwt.decode(token_user2, options={"verify_signature": False})
print(f"\n{decoded_token_user2}")
token_admin = get_user_token("example-admin", "changeme")
decoded_token_admin = jwt.decode(token_admin, options={"verify_signature": False})
print(f"\n{decoded_token_admin}")


{'exp': 1736860641, 'iat': 1736860341, 'jti': '3563606b-c59f-4c9a-bcbd-55e323e7eb70', 'iss': 'https://iam-auth.apx.develop.eoepca.org/realms/eoepca', 'aud': ['eoapi', 'ws-example-79', 'ws-example-78', 'account'], 'sub': '0965f986-de8e-43fa-ba1c-36d520eefe6f', 'typ': 'Bearer', 'azp': 'demo', 'session_state': 'd5d223ed-07cf-401d-ba4e-f7a5e1430825', 'acr': '1', 'realm_access': {'roles': ['offline_access', 'default-roles-eoepca', 'uma_authorization']}, 'resource_access': {'eoapi': {'roles': ['stac_editor']}, 'ws-example-79': {'roles': ['ws_access']}, 'ws-example-78': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'manage-account-links', 'view-profile']}}, 'scope': 'profile email', 'sid': 'd5d223ed-07cf-401d-ba4e-f7a5e1430825', 'email_verified': False, 'preferred_username': 'example-user-1'}

{'exp': 1736860641, 'iat': 1736860341, 'jti': '603c33f9-3e0e-4b0f-bdd4-dfcb81a7e530', 'iss': 'https://iam-auth.apx.develop.eoepca.org/realms/eoepca', 'aud': 'account', 'sub': '73bdf

In [5]:
print(decoded_token_user1["aud"])
print(decoded_token_user1["resource_access"])

['eoapi', 'ws-example-79', 'ws-example-78', 'account']
{'eoapi': {'roles': ['stac_editor']}, 'ws-example-79': {'roles': ['ws_access']}, 'ws-example-78': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'manage-account-links', 'view-profile']}}


### review existing workspaces

In [6]:
ws_user1 = [entry for entry in decoded_token_user1["aud"] if entry.startswith('ws-')]
print(f"user1: {ws_user1}")
ws_user2 = [entry for entry in decoded_token_user2["aud"] if entry.startswith('ws-')]
print(f"user2: {ws_user2}")

user1: ['ws-example-79', 'ws-example-78']
user2: []


In [7]:
def access_ws(ws_name, token):
    headers = {
        'Authorization': 'Bearer ' + token
    }
    url = f"{workspaces_endpoint}/{ws_name}"
    print(f"HTTP GET {url}")
    response = requests.get(url, headers=headers)
    print(response)
    print(response.text)
    return response

In [8]:
#manually add/remove users in keycloak and check access here
access_ws("ws-example-78", token_user1)

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-78
<Response [200]>
{"status":"ready","endpoints":[],"storage":{"credentials":{"AWS_ACCESS_KEY_ID":"ws-example-78","AWS_ENDPOINT_URL":"https://minio.develop.eoepca.org","AWS_REGION":"eoepca-demo","AWS_SECRET_ACCESS_KEY":"IBbyCDqxxlteTpZcZkV3sZSDzZBsAmXbFuYfczV5RYHSPeLPBv9MKClpsv2TBrF0","access":"ws-example-78","bucketname":"ws-example-78","secret":"IBbyCDqxxlteTpZcZkV3sZSDzZBsAmXbFuYfczV5RYHSPeLPBv9MKClpsv2TBrF0","endpoint":"https://minio.develop.eoepca.org","region":"eu-central-1"}},"container_registry":null}


<Response [200]>

### workspace creation

In [9]:
preferred_ws_name = f"example-{datetime.datetime.now().timestamp():.0f}"
preferred_ws_name

'example-1736860343'

In [10]:
response = requests.post(
    workspaces_endpoint,
    headers={
        #'Authorization': 'Bearer ' + token_user1
        'Authorization': 'Bearer ' + token_admin
    },
    json={
        "preferred_name": preferred_ws_name,
        "default_owner": "example-user-1"
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
ws_name = response.json()["name"]
print(f"created workspace '{ws_name}'")

created workspace 'ws-example-1736860343'


In [11]:
while True:
    time.sleep(3)
    response = access_ws(ws_name, get_user_token("example-user-1", "changeme"))
    if response.status_code == 200:
        try:
            workspace_data = response.json()
            if workspace_data.get("status") == "ready":
                print("ready")
                break
        except ValueError:
            print("not ready yet")
    
    print("...retrying")
    

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736860343
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736860343
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736860343
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736860343
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736860343
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET h

In [12]:
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]

### workspace s3 access 

In [13]:
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=minio_endpoint)

In [14]:
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Application package uploaded successfully


In [15]:
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(f"Key: {obj.key}, Size: {obj.size} bytes")

Key: application-package/s-expression/s-expression-0_0_2.cwl, Size: 2302 bytes


In [16]:
bucket.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '181A914FFB956970',
   'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Tue, 14 Jan 2025 13:12:59 GMT',
    'content-type': 'application/xml',
    'content-length': '201',
    'connection': 'keep-alive',
    'accept-ranges': 'bytes',
    'content-security-policy': 'block-all-mixed-content',
    'strict-transport-security': 'max-age=15724800; includeSubDomains',
    'vary': 'Origin, Accept-Encoding',
    'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
    'x-amz-request-id': '181A914FFB956970',
    'x-content-type-options': 'nosniff',
    'x-xss-protection': '1; mode=block'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'application-package/s-expression/s-expression-0_0_2.cwl'}]}]

### workspace deletion

In [17]:
response = requests.delete(
    f"{workspaces_endpoint}/{ws_name}",
    headers={
        'Authorization': 'Bearer ' + token_admin
    }
)
response.raise_for_status()
print(f"deleted workspace '{ws_name}'")

deleted workspace 'ws-example-1736860343'
